# Dataset augmentation

In [95]:
import zipfile
import os

# Path to the zip file and the extraction directory
zip_file_path = 'initial_annotation.zip'
extract_dir = 'initial_annotation'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete!")


Extraction complete!


In [96]:
import json

# Path to the annotations file
annotations_file = os.path.join(extract_dir, 'result.json')

# Load the annotations
with open(annotations_file, 'r') as f:
    coco_annotations = json.load(f)

print("Annotations loaded!")

Annotations loaded!


## Masking and cropping

In [105]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from typing import Union


def create_mask(segmentation, width, height):
    mask = Image.new('L', (width, height), 0)
    for polygon in segmentation:
        ImageDraw.Draw(mask).polygon(polygon, outline=1, fill=1)
    return mask

def crop_and_paste(coco_annotations, image_dir, background_image_path, output_dir, scale: float = 0.1, 
                   loc: tuple[float, float] = (0., 0.), output_size: tuple[int, int] = (300, 300), 
                   offset_callback: callable = lambda: (100, 100),
                   aug_idx: int = 0):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the background image
    background_image = Image.open(background_image_path)

    # Loop over each annotation
    for i_ann, annotation in enumerate(coco_annotations['annotations']):
        # Load the image
        image_info = next(image for image in coco_annotations['images'] if image['id'] == annotation['image_id'])
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = Image.open(image_path)
        # plt.figure()
        # plt.imshow(np.array(image))
        # plt.show()

        # Create a mask from the segmentation data
        segmentation = annotation['segmentation']
        width, height = image_info['width'], image_info['height']
        mask = create_mask(segmentation, width, height)
        # plt.figure()
        # plt.imshow(np.array(mask))
        # plt.show()

        # Apply the mask to the image to get the cropped object
        image_array = np.array(image)
        mask_array = np.array(mask)
        cropped_image_array = np.zeros_like(image_array)
        for i in range(3):  # For each color channel
            cropped_image_array[:, :, i] = image_array[:, :, i] * mask_array
        cropped_image = Image.fromarray(cropped_image_array)
        # plt.figure()
        # plt.imshow(np.array(cropped_image))
        # plt.show()

        # Restric the cropped image to the boundingbox
        bbox = [int(_) for _ in annotation['bbox']]
        bbox_cropped_image = Image.fromarray(np.array(cropped_image)[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]])
        bbox_mask = Image.fromarray(np.array(mask)[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]*255)
        # plt.figure()
        # plt.imshow(np.array(bbox_cropped_image))
        # plt.show()
        # plt.figure()
        # plt.imshow(np.array(bbox_mask))
        # plt.show()

        # Resize
        width = int(scale*background_image.width)
        height = int(bbox_cropped_image.height * (width/bbox_cropped_image.width))
        bbox_cropped_image = bbox_cropped_image.resize((width, height))
        bbox_mask = bbox_mask.resize((width, height))

        # Paste the cropped image onto the background image
        background_copy = background_image.copy()
        loc_in_bg = (int(loc[0]*background_copy.width), int(loc[1]*background_copy.height))
        background_copy.paste(bbox_cropped_image, loc_in_bg, bbox_mask)
        offset = offset_callback()
        pasted = background_copy.crop((loc_in_bg[0]-offset[0], loc_in_bg[1]-offset[1], 
                                       loc_in_bg[0]-offset[0]+output_size[1], loc_in_bg[1]-offset[1]+output_size[1]))
        # plt.figure()
        # plt.imshow(np.array(pasted))
        # plt.show()

        # Generate mask
        mask_copy = Image.fromarray(0*np.array(background_image.copy()))
        mask_copy.paste(bbox_mask, loc_in_bg, bbox_mask)
        mask_pasted = mask_copy.crop((loc_in_bg[0]-offset[0], loc_in_bg[1]-offset[1], 
                                       loc_in_bg[0]-offset[0]+output_size[1], loc_in_bg[1]-offset[1]+output_size[1]))
        # plt.figure()
        # plt.imshow(np.array(mask_pasted))
        # plt.show()

        # Save the resulting image
        name = os.path.split(image_info['file_name'])[-1]
        output_path = os.path.join(output_dir, f"pasted_{aug_idx}_{i_ann}_{name}")
        pasted.save(output_path)

        # Save mask
        name = os.path.split(image_info['file_name'])[-1]
        output_path = os.path.join(output_dir, f"mask_{aug_idx}_{i_ann}_{name}")
        mask_pasted.save(output_path)

In [112]:
# Paths
image_dir = 'initial_annotation'
output_dir = 'pasted'

background_image_paths = [f'backgrounds/bg{_+1}.jpg' for _ in range(6)]
scales = [0.10, 0.15, 0.20]
locs = [(0.1, 0.1), (0.1, 0.3), (0.1, 0.5),
        (0.3, 0.1), (0.3, 0.3), (0.3, 0.5),
        (0.5, 0.1), (0.5, 0.3), (0.5, 0.5)]

# Crop and paste images
aug_idx = 0
for background_image_path in background_image_paths:
    for scale in scales:
        for loc in locs:
            offset_callback = lambda: (int(np.random.uniform(0,128)), int(np.random.uniform(0,128)))
            crop_and_paste(coco_annotations, image_dir, background_image_path, output_dir, scale=scale, loc=loc, 
                           output_size=(256,256), offset_callback=offset_callback, aug_idx=aug_idx)
            aug_idx += 1